# Importing Dependencies

In [36]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import f1_score, classification_report, accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate


In [37]:
#Reading the data
df = pd.read_csv('concated.csv')

In [3]:
df.head()

,Unnamed: 0,HotelName,HotelPrice,HotelAddress,HotelRatings,AuthorName,Authorlocation,Review,title,Service,Cleanliness,Overall,Value,Sleep Quality,Rooms,Location,"Business service (e.g., internet access)",Check in / front desk
0,0,Hilton Garden Inn Baltimore White Marsh,$135 - $193*,"5015 Campbell Blvd, Baltimore, MD 21236","{'Service': '5', 'Cleanliness': '5', 'Overall'...",Nadine R,"Jacksonville, Florida","Usually stay near the airport, but this trip w...",“Great place and location”,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
1,1,Hilton Garden Inn Baltimore White Marsh,$135 - $193*,"5015 Campbell Blvd, Baltimore, MD 21236","{'Service': '4', 'Cleanliness': '4', 'Overall'...",blt3116,"Chillicothe, Ohio",Stayed at this Hilton for 2 nights. It was lik...,"“Nice stay, nice surroundings”",4.0,4.0,4.0,4.0,4.0,4.0,5.0,NaN,NaN
2,2,Hilton Garden Inn Baltimore White Marsh,$135 - $193*,"5015 Campbell Blvd, Baltimore, MD 21236","{'Service': '5', 'Cleanliness': '5', 'Overall'...",ExSpec5,Northern Virginia,"Stayed there one night, December 16, on the wa...",“Perfect for an overnight just off I-95”,5.0,5.0,4.0,3.0,3.0,5.0,5.0,NaN,NaN
3,3,Hilton Garden Inn Baltimore White Marsh,$135 - $193*,"5015 Campbell Blvd, Baltimore, MD 21236","{'Service': '5', 'Cleanliness': '5', 'Overall'...",kevinlynch3,Raleigh,I just stayed here last weekend and have alrea...,“Great Hotel”,5.0,5.0,5.0,4.0,5.0,5.0,5.0,NaN,NaN
4,4,Hilton Garden Inn Baltimore White Marsh,$135 - $193*,"5015 Campbell Blvd, Baltimore, MD 21236","{'Service': '5', 'Cleanliness': '5', 'Overall'...",nobumponalog,"Belmont, MA",My mother who is 90 and I stayed one night on ...,“good room for handicapped person”,5.0,5.0,5.0,4.0,5.0,5.0,5.0,NaN,NaN


In [38]:
#Droping the unwanted columns
df.drop(['HotelRatings','HotelAddress','HotelName','Unnamed: 0','Authorlocation','AuthorName'],axis=1,inplace=True)

In [39]:
df.head()

,HotelPrice,Review,title,Service,Cleanliness,Overall,Value,Sleep Quality,Rooms,Location,"Business service (e.g., internet access)",Check in / front desk
0,$135 - $193*,"Usually stay near the airport, but this trip w...",“Great place and location”,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
1,$135 - $193*,Stayed at this Hilton for 2 nights. It was lik...,"“Nice stay, nice surroundings”",4.0,4.0,4.0,4.0,4.0,4.0,5.0,NaN,NaN
2,$135 - $193*,"Stayed there one night, December 16, on the wa...",“Perfect for an overnight just off I-95”,5.0,5.0,4.0,3.0,3.0,5.0,5.0,NaN,NaN
3,$135 - $193*,I just stayed here last weekend and have alrea...,“Great Hotel”,5.0,5.0,5.0,4.0,5.0,5.0,5.0,NaN,NaN
4,$135 - $193*,My mother who is 90 and I stayed one night on ...,“good room for handicapped person”,5.0,5.0,5.0,4.0,5.0,5.0,5.0,NaN,NaN


In [40]:
#Checking the null values
(df.isnull().sum(axis=0)/len(df))*100

HotelPrice                                   0.000000
Review                                       0.000000
title                                        0.000000
Service                                      9.054203
Cleanliness                                  9.181028
Overall                                      0.000000
Value                                        9.029691
Sleep Quality                               47.947068
Rooms                                       17.773840
Location                                    22.571790
Business service (e.g., internet access)    89.300052
Check in / front desk                       82.523484
dtype: float64

In [7]:
#Dropingcolumns having null values with high percentage
df.drop(['Sleep Quality','Business service (e.g., internet access)','Check in / front desk'],axis=1,inplace=True)

In [8]:
df["ReviewDetails"]=df['title']+" "+df["Review"]

In [9]:
df.head()

,HotelPrice,Review,title,Service,Cleanliness,Overall,Value,Rooms,Location,ReviewDetails
0,$135 - $193*,"Usually stay near the airport, but this trip w...",“Great place and location”,5.0,5.0,5.0,5.0,5.0,5.0,“Great place and location” Usually stay near t...
1,$135 - $193*,Stayed at this Hilton for 2 nights. It was lik...,"“Nice stay, nice surroundings”",4.0,4.0,4.0,4.0,4.0,5.0,"“Nice stay, nice surroundings” Stayed at this ..."
2,$135 - $193*,"Stayed there one night, December 16, on the wa...",“Perfect for an overnight just off I-95”,5.0,5.0,4.0,3.0,5.0,5.0,“Perfect for an overnight just off I-95” Staye...
3,$135 - $193*,I just stayed here last weekend and have alrea...,“Great Hotel”,5.0,5.0,5.0,4.0,5.0,5.0,“Great Hotel” I just stayed here last weekend ...
4,$135 - $193*,My mother who is 90 and I stayed one night on ...,“good room for handicapped person”,5.0,5.0,5.0,4.0,5.0,5.0,“good room for handicapped person” My mother w...


####  Stop words  removal and  Stemming

In [20]:

stemmer = PorterStemmer()
words=stopwords.words("english")
df['cleaned'] = df['ReviewDetails'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [21]:
df['cleaned'].head()

0    great place locat usual stay near airport trip...
1    nice stay nice surround stay hilton night it l...
2    perfect overnight i stay one night decemb way ...
3    great hotel i stay last weekend alreadi plan s...
4    good room handicap person my mother i stay one...
Name: cleaned, dtype: object

In [ ]:
df['Cleaned_with_price'] = df['cleaned']+" "+df['HotelPrice']

In [ ]:
df['Cleaned_with_price'].to_pickle("cleaned.pkl")

In [9]:
df_cleaned=pd.read_pickle("cleaned.pkl")

In [10]:
df['Overall'] =df['Overall'].astype('str')

#### Train and test split

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(df_cleaned,df['Overall'], test_size = 0.10, random_state = 42)

# TF-IDF 

In [13]:
#TF-IDF on Training set
vectorizer=TfidfVectorizer(min_df=0.1,stop_words="english",sublinear_tf=True,ngram_range=(1,1),norm='l2')
final_features_train = vectorizer.fit(X_train)
final_features_train = vectorizer.transform(X_train)
final_features_train.shape

(1329155, 112)

In [78]:
#TF-IDF on Test set
vectorizer=TfidfVectorizer(min_df=0.1,stop_words="english",sublinear_tf=True,ngram_range=(1,1),norm='l2')
vectorizer=vectorizer.fit(X_test)
final_features_test = vectorizer.transform(X_test)
final_features_test.shape

(147684, 112)

# XgBoost Model ,Training and testing

In [85]:
RANDOM_STATE = 100

In [86]:
xgb_model = xgb.XGBClassifier(max_depth=10, random_state=RANDOM_STATE,
learning_rate=0.1, colsample_bytree=.7, gamma=0, alpha=0,objective='multi:softmax', eta=0.3,subsample=0.8).fit(final_features_train, Y_train) 

In [87]:
xgb_prediction = xgb_model.predict(final_features_train)

In [44]:
xgb_prediction = xgb_model.predict(final_features_test)

# Model evaluation

##### Training score

In [88]:
print('training score:', f1_score(Y_train, xgb_model.predict(final_features_train), average='macro'))

training score: 0.48326342993590926


In [42]:
print(classification_report(Y_train, xgb_model.predict(final_features_train)))

C:\Users\ayu_a\.conda\envs\workspace-gpu\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         1
         1.0       0.61      0.71      0.66     89294
         2.0       0.70      0.29      0.41     93285
         3.0       0.61      0.35      0.45    175595
         4.0       0.60      0.63      0.62    417448
         5.0       0.71      0.84      0.77    553532

    accuracy                           0.66   1329155
   macro avg       0.54      0.47      0.48   1329155
weighted avg       0.66      0.66      0.65   1329155



##### Testing score

In [46]:
print('validation score:', f1_score(Y_test, xgb_prediction, average='macro'))
print(classification_report(Y_test, xgb_prediction))

validation score: 0.49717569867088107
              precision    recall  f1-score   support

         1.0       0.56      0.63      0.60      9879
         2.0       0.49      0.17      0.26     10133
         3.0       0.49      0.27      0.35     19526
         4.0       0.54      0.57      0.55     46202
         5.0       0.68      0.80      0.73     61944

    accuracy                           0.60    147684
   macro avg       0.55      0.49      0.50    147684
weighted avg       0.59      0.60      0.58    147684



In [50]:
accuracy = accuracy_score(Y_test, xgb_prediction)
accuracy

0.6042428428265757

In [48]:
#Confusion Matrix
confusion_matrix(Y_test, xgb_prediction)

array([[ 6257,   743,   927,   752,  1200],
       [ 2370,  1757,  2093,  2306,  1607],
       [ 1318,   699,  5247,  8665,  3597],
       [  518,   265,  1817, 26247, 17355],
       [  653,   143,   650, 10769, 49729]], dtype=int64)

# Test with a new review.

In [79]:
new_review = ['The room was dump; there was a small wardrobe which we could not use as it was not pleasant (to me). I will have rather place a open wardrobe.']
#TF-IDF on Test set
final_features_new = vectorizer.transform(new_review)
type(final_features_new)
#final_features_new=final_features_new.toarray()
#final_features_new.shape

scipy.sparse.csr.csr_matrix

In [80]:
final_features_new.shape

(1, 112)

In [82]:
pred = xgb_model.predict(final_features_new)
#labels = [1,2,3,4,5]
print("rating for given Review : ",pred)

['1.0']
